# The Battle of Neighborhoods

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

## Download and Explore Dataset

Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

This dataset exists for free on the web.
The link to the dataset is: https://geo.nyu.edu/catalog/nyu_2451_34572

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
# Load the data
import json # library to handle JSON files
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
# Explore the data
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [5]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

## Transform the data into a pandas dataframe

In [6]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [7]:
# Then let's loop through the data and fill the dataframe one row at a time.
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [8]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [9]:
# Make sure the dataset has all 5 boroughs and 306 neighborhoods.
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


## Use geopy library to get the latitude and longitude values of New York City.

In [10]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


## Create a map of New York with neighborhoods superimposed on top.

In [11]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [12]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [13]:
# Get the geographical coordinates of Manhattan.
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [14]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

## Define Foursquare Credentials and Version

In [15]:
CLIENT_ID = 'ZFBMQTKP23WOBO4VKTXWPD1BU5FVBZ2YXEY4DDDDQYXAIADN' # your Foursquare ID
CLIENT_SECRET = 'HPND5IUIC1DGINWG4SED3ID5P555Z3P3142GELPHQW12SSXR' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZFBMQTKP23WOBO4VKTXWPD1BU5FVBZ2YXEY4DDDDQYXAIADN
CLIENT_SECRET:HPND5IUIC1DGINWG4SED3ID5P555Z3P3142GELPHQW12SSXR


In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?client_id=ZFBMQTKP23WOBO4VKTXWPD1BU5FVBZ2YXEY4DDDDQYXAIADN&client_secret=HPND5IUIC1DGINWG4SED3ID5P555Z3P3142GELPHQW12SSXR&v=20180602&ll=40.73,-73.01&query=food%20category'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,  
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


## Check the size of the resulting dataframe

In [18]:
print(manhattan_venues.shape)
manhattan_venues.head()

(1200, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,PeraBell Food Bar,40.765781,-73.013731,American Restaurant
1,Marble Hill,40.876551,-73.91066,Aldi Food Market,40.778311,-73.033230,Supermarket
2,Marble Hill,40.876551,-73.91066,Delfiore Pizza & Food Co.,40.765692,-73.013344,Italian Restaurant
3,Marble Hill,40.876551,-73.91066,Best Meal Chinese Food,40.765382,-73.013084,Chinese Restaurant
4,Marble Hill,40.876551,-73.91066,Smith Haven Mall Food Court,40.863369,-73.129668,Food Court


In [19]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,30,30,30,30,30,30
Carnegie Hill,30,30,30,30,30,30
Central Harlem,30,30,30,30,30,30
Chelsea,30,30,30,30,30,30
Chinatown,30,30,30,30,30,30
Civic Center,30,30,30,30,30,30
Clinton,30,30,30,30,30,30
East Harlem,30,30,30,30,30,30
East Village,30,30,30,30,30,30


In [20]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,American Restaurant,Butcher,Chinese Restaurant,Deli / Bodega,Food Court,Food Truck,Frozen Yogurt Shop,Gourmet Shop,Grocery Store,Italian Restaurant,Museum,Organic Grocery,Other Repair Shop,Restaurant,Southern / Soul Food Restaurant,Supermarket
0,Marble Hill,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,Marble Hill,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,Marble Hill,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [21]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,American Restaurant,Butcher,Chinese Restaurant,Deli / Bodega,Food Court,Food Truck,Frozen Yogurt Shop,Gourmet Shop,Grocery Store,Italian Restaurant,Museum,Organic Grocery,Other Repair Shop,Restaurant,Southern / Soul Food Restaurant,Supermarket
0,Battery Park City,0.066667,0.033333,0.066667,0.033333,0.166667,0.266667,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.066667
1,Carnegie Hill,0.066667,0.033333,0.066667,0.033333,0.166667,0.266667,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.066667
2,Central Harlem,0.066667,0.033333,0.066667,0.033333,0.166667,0.266667,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.066667
3,Chelsea,0.066667,0.033333,0.066667,0.033333,0.166667,0.266667,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.066667
4,Chinatown,0.066667,0.033333,0.066667,0.033333,0.166667,0.266667,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.066667
5,Civic Center,0.066667,0.033333,0.066667,0.033333,0.166667,0.266667,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.066667
6,Clinton,0.066667,0.033333,0.066667,0.033333,0.166667,0.266667,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.066667
7,East Harlem,0.066667,0.033333,0.066667,0.033333,0.166667,0.266667,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.066667
8,East Village,0.066667,0.033333,0.066667,0.033333,0.166667,0.266667,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.066667
9,Financial District,0.066667,0.033333,0.066667,0.033333,0.166667,0.266667,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.066667


In [22]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
                 venue  freq
0           Food Truck  0.27
1           Food Court  0.17
2  American Restaurant  0.07
3   Chinese Restaurant  0.07
4          Supermarket  0.07


----Carnegie Hill----
                 venue  freq
0           Food Truck  0.27
1           Food Court  0.17
2  American Restaurant  0.07
3   Chinese Restaurant  0.07
4          Supermarket  0.07


----Central Harlem----
                 venue  freq
0           Food Truck  0.27
1           Food Court  0.17
2  American Restaurant  0.07
3   Chinese Restaurant  0.07
4          Supermarket  0.07


----Chelsea----
                 venue  freq
0           Food Truck  0.27
1           Food Court  0.17
2  American Restaurant  0.07
3   Chinese Restaurant  0.07
4          Supermarket  0.07


----Chinatown----
                 venue  freq
0           Food Truck  0.27
1           Food Court  0.17
2  American Restaurant  0.07
3   Chinese Restaurant  0.07
4          Supermarket  0.07


----Civic Cente

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Food Truck,Food Court,Supermarket,Chinese Restaurant,American Restaurant,Southern / Soul Food Restaurant,Restaurant,Other Repair Shop,Organic Grocery,Museum
1,Carnegie Hill,Food Truck,Food Court,Supermarket,Chinese Restaurant,American Restaurant,Southern / Soul Food Restaurant,Restaurant,Other Repair Shop,Organic Grocery,Museum
2,Central Harlem,Food Truck,Food Court,Supermarket,Chinese Restaurant,American Restaurant,Southern / Soul Food Restaurant,Restaurant,Other Repair Shop,Organic Grocery,Museum
3,Chelsea,Food Truck,Food Court,Supermarket,Chinese Restaurant,American Restaurant,Southern / Soul Food Restaurant,Restaurant,Other Repair Shop,Organic Grocery,Museum
4,Chinatown,Food Truck,Food Court,Supermarket,Chinese Restaurant,American Restaurant,Southern / Soul Food Restaurant,Restaurant,Other Repair Shop,Organic Grocery,Museum


## Cluster Neighborhoods

In [25]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [26]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,0,Food Truck,Food Court,Supermarket,Chinese Restaurant,American Restaurant,Southern / Soul Food Restaurant,Restaurant,Other Repair Shop,Organic Grocery,Museum
1,Manhattan,Chinatown,40.715618,-73.994279,0,Food Truck,Food Court,Supermarket,Chinese Restaurant,American Restaurant,Southern / Soul Food Restaurant,Restaurant,Other Repair Shop,Organic Grocery,Museum
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Food Truck,Food Court,Supermarket,Chinese Restaurant,American Restaurant,Southern / Soul Food Restaurant,Restaurant,Other Repair Shop,Organic Grocery,Museum
3,Manhattan,Inwood,40.867684,-73.921210,0,Food Truck,Food Court,Supermarket,Chinese Restaurant,American Restaurant,Southern / Soul Food Restaurant,Restaurant,Other Repair Shop,Organic Grocery,Museum
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Food Truck,Food Court,Supermarket,Chinese Restaurant,American Restaurant,Southern / Soul Food Restaurant,Restaurant,Other Repair Shop,Organic Grocery,Museum


In [27]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [28]:
# Cluster 1
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Food Truck,Food Court,Supermarket,Chinese Restaurant,American Restaurant,Southern / Soul Food Restaurant,Restaurant,Other Repair Shop,Organic Grocery,Museum
1,Chinatown,Food Truck,Food Court,Supermarket,Chinese Restaurant,American Restaurant,Southern / Soul Food Restaurant,Restaurant,Other Repair Shop,Organic Grocery,Museum
2,Washington Heights,Food Truck,Food Court,Supermarket,Chinese Restaurant,American Restaurant,Southern / Soul Food Restaurant,Restaurant,Other Repair Shop,Organic Grocery,Museum
3,Inwood,Food Truck,Food Court,Supermarket,Chinese Restaurant,American Restaurant,Southern / Soul Food Restaurant,Restaurant,Other Repair Shop,Organic Grocery,Museum
4,Hamilton Heights,Food Truck,Food Court,Supermarket,Chinese Restaurant,American Restaurant,Southern / Soul Food Restaurant,Restaurant,Other Repair Shop,Organic Grocery,Museum
5,Manhattanville,Food Truck,Food Court,Supermarket,Chinese Restaurant,American Restaurant,Southern / Soul Food Restaurant,Restaurant,Other Repair Shop,Organic Grocery,Museum
6,Central Harlem,Food Truck,Food Court,Supermarket,Chinese Restaurant,American Restaurant,Southern / Soul Food Restaurant,Restaurant,Other Repair Shop,Organic Grocery,Museum
7,East Harlem,Food Truck,Food Court,Supermarket,Chinese Restaurant,American Restaurant,Southern / Soul Food Restaurant,Restaurant,Other Repair Shop,Organic Grocery,Museum
8,Upper East Side,Food Truck,Food Court,Supermarket,Chinese Restaurant,American Restaurant,Southern / Soul Food Restaurant,Restaurant,Other Repair Shop,Organic Grocery,Museum
9,Yorkville,Food Truck,Food Court,Supermarket,Chinese Restaurant,American Restaurant,Southern / Soul Food Restaurant,Restaurant,Other Repair Shop,Organic Grocery,Museum
